## Boundary detection

In [6]:
import numpy as np
import cv2
from matplotlib import pyplot as plt


In [7]:

MIN_MATCH_COUNT = 10

obj_img = cv2.imread('ambientcommonsbook.jpg',0)          # queryImage

ret, frame = cv2.VideoCapture(0).read()

#scene_img = cv2.imread('books.jpg',0) # trainImage

scene_img = frame


orb = cv2.ORB_create(nfeatures=2000)


In [ ]:
dir(cv2)

In [8]:
def find_object(img1, img2):

    # find the keypoints and descriptors with SIFT
    kp1, des1 = orb.detectAndCompute(img1,None)
    kp2, des2 = orb.detectAndCompute(img2,None)

    FLANN_INDEX_KDTREE = 0
    FLANN_INDEX_LSH = 6

    index_params = dict(algorithm = FLANN_INDEX_LSH,
                       table_number = 6, # 12
                       key_size = 12,     # 20
                       multi_probe_level = 1) #2

    search_params = dict(checks = 50)

    flann = cv2.FlannBasedMatcher(index_params, search_params)

    matches = flann.knnMatch(des1, des2, k=2)

    #matches= flann.knnMatch(np.asarray(des1,np.float32),np.asarray(des2,np.float32), k=2)


    # store all the good matches as per Lowe's ratio test.
    good = []
    for match in matches:
        if(len(match) == 2):
            m,n = match
            if m.distance < 0.7*n.distance:
                good.append(m)
                    
            
    if len(good)>MIN_MATCH_COUNT:
        src_pts = np.float32([ kp1[m.queryIdx].pt for m in good ]).reshape(-1,1,2)
        dst_pts = np.float32([ kp2[m.trainIdx].pt for m in good ]).reshape(-1,1,2)

        M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC,5.0)
        matchesMask = mask.ravel().tolist()

        h,w = img1.shape
        pts = np.float32([ [0,0],[0,h-1],[w-1,h-1],[w-1,0] ]).reshape(-1,1,2)
        
        dst = cv2.perspectiveTransform(pts,M)
        
#        img2 = cv2.polylines(img2,[np.int32(dst)],True,255,3, cv2.LINE_AA)
        
        return dst
    else:
        #print("Not enough matches are found - {}/{}".format(len(good),MIN_MATCH_COUNT))
        matchesMask = None
        return None



In [5]:
coords = find_object(obj_img, scene_img)


if(coords.any()):
    draw_params = dict(matchColor = (0,255,0), # draw matches in green color
               singlePointColor = None,
               flags = 2)

    img2 = cv2.polylines(scene_img,[np.int32(coords)],True,255,3, cv2.LINE_AA)
#    img3 = cv2.drawMatches(obj_img,kp1,scene_img,kp2,good,None,**draw_params)

    plt.imshow(img2, 'gray'),plt.show()

AttributeError: 'NoneType' object has no attribute 'any'

## Boundary detection with camera

In [14]:
import numpy as np
import cv2


In [11]:
ret, frame = cv2.VideoCapture(0).read()
cap.release()
frame_height, frame_width = frame.shape[:2]

def blank_overlay(height, width):
    return np.zeros((height, width, 3), dtype=np.uint8)

def rectangle_creation(event, x, y, flags, param):
    global temp_point, rectangles
    # if the left mouse button was clicked, record the starting
    if event == cv2.EVENT_LBUTTONDOWN:
        temp_point = (x, y)
    # check to see if the left mouse button was released
    elif event == cv2.EVENT_LBUTTONUP:
        if(temp_point):
            rectangles = [[temp_point, (x, y)]] # we only use one for now
            recalculate_region(rectangles[0])    
        temp_point = None
        
def recalculate_region(r):
    global rectangles, overlay, feature_region, frame, kp_fr, des_fr, frame_height, frame_width
    
    print(r[0], r[1])
    
    overlay = blank_overlay(frame_height, frame_width)
    cv2.rectangle(overlay, r[0], r[1], (0, 255, 0), 2)
    
    feature_region = frame[r[0][1]:r[1][1], r[0][0]:r[1][0]]
    
    kp_fr, des_fr = orb.detectAndCompute(feature_region, None)
    
    feature_region = cv2.drawKeypoints(frame[r[0][1]:r[1][1], r[0][0]:r[1][0]], kp_fr, None, color=(0,255,0),  flags=cv2.DrawMatchesFlags_DEFAULT)
    

# now let's initialize the list of reference point
temp_point = []
rectangles = [] 
feature_region = np.zeros((1, 1, 3), dtype=np.uint8)

overlay = blank_overlay(frame_height, frame_width)

cv2.namedWindow("image")
cv2.setMouseCallback("image", rectangle_creation)


In [15]:
obj_img = cv2.imread('attentionbook.jpg',0) 
cap = cv2.VideoCapture(0)

alpha = 0.5

while True:
    # Capture frame-by-frame
    ret, frame = cap.read()

    coords = find_object(obj_img, frame)


    if(coords is not None and coords.any()):
        cv2.addWeighted(overlay, alpha, frame, 1 - alpha, 0, frame)

        draw_params = dict(matchColor = (0,255,0), # draw matches in green color
                   singlePointColor = None,
                   flags = 2)

        img2 = cv2.polylines(frame,[np.int32(coords)],True,255,3, cv2.LINE_AA)
    #    img3 = cv2.drawMatches(obj_img,kp1,scene_img,kp2,good,None,**draw_params)
        cv2.imshow("image", img2)

        
        
    key = cv2.waitKey(1) & 0xFF
    # press 'r' to reset the window
    if key == ord("r"):
        overlay = new_overlay()
    # if the 'c' key is pressed, break from the loop
    if key == ord("c"):
        break


# When everything done, release the captureqq
cap.release()
cv2.destroyAllWindows()

KeyboardInterrupt: 